## Context

- analysis in PAL014 identified Stim-Unstim axis as the strongest source of variation, but not much else
- there's some evidence from Sam's factor analysis that there may be subtle effects at play that are not getting captured by the psuedobulk-PCA methods I used
- I'm going to try different methods to try and extract any variance I can
    - miloDE
    - SCVI

## Libraries and Settings

In [7]:
import scanpy as sc
import mudata as md
import rapids_singlecell as rsc
import torch

## Data Loading

In [6]:
mdata = md.read_h5mu("/lustre/fs4/cao_lab/scratch/plaffont/personal_datasets/CTR02/ctr02.h5mu")
mdata

MuData object with n_obs × n_vars = 42903 × 32303
  var:	'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence', 'target_gene_id', 'target_gene_name'
  3 modalities
    rna:	42903 x 32286
      var:	'gene_ids', 'feature_types', 'genome'
    guide:	42903 x 12
      var:	'gene_ids', 'feature_types', 'pattern', 'read', 'sequence', 'target_gene_id', 'target_gene_name'
    cite:	42903 x 5
      var:	'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'

In [8]:
mdata["guide"].var

,gene_ids,feature_types,pattern,read,sequence,target_gene_id,target_gene_name
NTC.1,NTC.1,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,TACGCGACGACGTATCAGGT,Non-Targeting,Non-Targeting
NTC.2,NTC.2,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,CGCGCGTAATCGGTCGTTTC,Non-Targeting,Non-Targeting
NCC.1,NCC.1,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,ATGTATCGCGAGTACATACT,Non-Targeting,Non-Targeting
NCC.2,NCC.2,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,TATAGGTTATGACGCGCTGC,Non-Targeting,Non-Targeting
Ifngr1.1,Ifngr1.1,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,TCGGGTAATCTCTCCGAGCC,ENSMUSG00000020009,Ifngr1
Ifngr1.2,Ifngr1.2,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,TATGTTTCGTATGTAGGATC,ENSMUSG00000020009,Ifngr1
Ifngr1.3,Ifngr1.3,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,GGTATTTGCGTATTGGTATA,ENSMUSG00000020009,Ifngr1
Ifngr1.4,Ifngr1.4,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,GATATTGTATTTCAGTAGAC,ENSMUSG00000020009,Ifngr1
Tgfbr2.1,Tgfbr2.1,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,GGGGCATCGCTCATCTCCAC,ENSMUSG00000032440,Tgfbr2
Tgfbr2.2,Tgfbr2.2,CRISPR Guide Capture,TTCCAGCATAGCTCTTAAAC(BC),R2,ATCGAGCTGGACACGCTGGT,ENSMUSG00000032440,Tgfbr2
